In [1]:
import pandas as pd

df = pd.read_csv('../data/train.csv')
df = df.head(5000)
# assign "No Value" value to each response of each row
df['response'] = 'No Value'
# df.to_csv('../data/small-train.csv', index=False)


In [ ]:
type(df.iloc[0].response)

In [4]:
import json
import re

def clean_text(text: str) -> list[str]:
    cleaned_text = re.sub(
    r'[\uD800-\uDBFF][\uDC00-\uDFFF]',
    lambda m: f'\\u{ord(m.group()):04x}',
    text)
    return json.loads(cleaned_text)

def split_lists(row):
    prompts = clean_text(row['prompt'])
    response_as = clean_text(row['response_a'])
    response_bs = clean_text(row['response_b'])
    # response = clean_text(row['response'])
    
    new_rows = []
    for prompt, response_a, response_b in zip(prompts, response_as, response_bs):
        new_row = row.copy()
        new_row['prompt'] = prompt
        new_row['response_a'] = response_a
        new_row['response_b'] = response_b
        new_rows.append(new_row)
    
    return pd.DataFrame(new_rows)

# Apply the function to each row and concatenate the results
split_df = pd.concat([split_lists(row) for _, row in df.iterrows()], ignore_index=True)
split_df.to_csv('../data/small-train-5000.csv', index=False)

In [31]:
import pandas as pd
df = pd.read_csv('../data/small-train-nemotron-4-340b_1.csv')

In [32]:
error_list = ['API REQUEST ERROR', 'NETWORK ERROR DUE TO HIGH TRAFFIC']

In [33]:
for index, row in df.iterrows():
    if any(error in row['response'] for error in error_list):
        df.at[index, 'response'] = 'No Value'
        print(index)

In [30]:
df.to_csv('../data/small-train-nemotron-4-340b_1.csv', index=False)